<a href="https://colab.research.google.com/github/koryca/mec-final-project/blob/main/Prototype.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install fastai -q --upgrade

     |████████████████████████████████| 188 kB 7.8 MB/s 
     |████████████████████████████████| 56 kB 2.0 MB/s 


In [2]:
from fastai.vision.all import *

In [3]:
import numpy as np
import pandas as pd
import os
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt

In [5]:
# create the .kaggle directory and an empty kaggle.json file
!mkdir -p /root/.kaggle
!touch /root/.kaggle/kaggle.json
!chmod 600 /root/.kaggle/kaggle.json

In [6]:
import json
kaggle_username = "korycallanan"
kaggle_key = "dcbf94c39955cae79de6516d1606d186"

with open("/root/.kaggle/kaggle.json", "w") as f:
    f.write(json.dumps({"username": kaggle_username, "key": kaggle_key}))

In [7]:
!kaggle datasets download -d datamunge/sign-language-mnist
!unzip -o sign-language-mnist.zip

 78% 49.0M/62.6M [00:00<00:00, 87.6MB/s]
100% 62.6M/62.6M [00:00<00:00, 102MB/s] 
Archive:  sign-language-mnist.zip
  inflating: amer_sign2.png          
  inflating: amer_sign3.png          
  inflating: american_sign_language.PNG  
  inflating: sign_mnist_test.csv     
  inflating: sign_mnist_test/sign_mnist_test.csv  
  inflating: sign_mnist_train.csv    
  inflating: sign_mnist_train/sign_mnist_train.csv  


In [31]:
path = Path('sign-language-mnist')
path.ls()

(#8) [Path('sign-language-mnist/sign_mnist_test'),Path('sign-language-mnist/sign_mnist_train'),Path('sign-language-mnist/sign-language-mnist.zip'),Path('sign-language-mnist/sign_mnist_train.csv'),Path('sign-language-mnist/amer_sign3.png'),Path('sign-language-mnist/sign_mnist_test.csv'),Path('sign-language-mnist/american_sign_language.PNG'),Path('sign-language-mnist/amer_sign2.png')]

In [34]:
TRAIN_DIR = path/'sign_mnist_train'/'sign_mnist_train.csv'
TEST_DIR = path/'sign_mnist_test'/'sign_mnist_test.csv'

In [35]:
train = pd.read_csv(TRAIN_DIR)
test = pd.read_csv(TEST_DIR)

In [36]:
train.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,3,107,118,127,134,139,143,146,150,153,...,207,207,207,207,206,206,206,204,203,202
1,6,155,157,156,156,156,157,156,158,158,...,69,149,128,87,94,163,175,103,135,149
2,2,187,188,188,187,187,186,187,188,187,...,202,201,200,199,198,199,198,195,194,195
3,2,211,211,212,212,211,210,211,210,210,...,235,234,233,231,230,226,225,222,229,163
4,13,164,167,170,172,176,179,180,184,185,...,92,105,105,108,133,163,157,163,164,179


In [64]:
def get_images(df):
    IMG_WIDTH =28
    IMG_HEIGHT=28
    
    df['img'] = df[df.columns[df.columns.str.startswith('pixel')]].apply(
        lambda x: PILImage(Image.fromarray(np.array(x.values).reshape((IMG_WIDTH,IMG_HEIGHT)).astype(np.uint8))),
        axis=1
    )
    
    return df[df.columns[[not x for x in df.columns.str.startswith('pixel')]]]

In [66]:
train_df = get_images(train)

In [67]:
def get_x(r): return r['img']
def get_y(r): return r['label']
dblock = DataBlock(blocks=(ImageBlock, CategoryBlock), get_x = get_x, get_y=get_y)
dls = dblock.dataloaders(train_df)

In [68]:
learn = cnn_learner(dls, resnet34, metrics=error_rate).to_fp16()
learn.fine_tune(10)


Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth


  0%|          | 0.00/83.3M [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
/usr/local/lib/python3.7/dist-packages/torch/cuda/amp/grad_scaler.py:115: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


epoch,train_loss,valid_loss,error_rate,time
0,1.254531,0.610491,0.178110,04:53


epoch,train_loss,valid_loss,error_rate,time
0,0.228566,0.051287,0.010563,09:12
1,0.067177,0.013824,0.004007,09:24
2,0.036834,0.006001,0.001821,09:23
3,0.047208,0.002340,0.000546,09:24
4,0.019912,0.007107,0.001457,09:31
5,0.006812,0.000628,0.000546,09:31
6,0.003795,0.000236,0.000000,09:27
7,0.001791,0.000046,0.000000,09:24
8,0.000880,0.000010,0.000000,09:31
9,0.001300,0.000005,0.000000,09:29
